# Global Alignment using Pandas

Here I have used the pandas library to iterate throught the nucleotide sequences to acheive the best possible alignment. It provides the score matrix and the best possible alignment.

In [15]:
import pandas as pd
def global_alignment(seq1, seq2, match_score=1, mismatch_score=-1, gap_penalty=-1):
    # Initialize the score matrix
    score_matrix = [[0] * (len(seq2) + 1) for _ in range(len(seq1) + 1)]

    # Initialize the traceback matrix
    traceback = [[0] * (len(seq2) + 1) for _ in range(len(seq1) + 1)]

    # Initialize the first row and column
    for i in range(1, len(seq1) + 1):
        score_matrix[i][0] = gap_penalty * i
        traceback[i][0] = 'U'
    for j in range(1, len(seq2) + 1):
        score_matrix[0][j] = gap_penalty * j
        traceback[0][j] = 'L'

    # Fill in the score and traceback matrices
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            match = score_matrix[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_score)
            delete = score_matrix[i - 1][j] + gap_penalty
            insert = score_matrix[i][j - 1] + gap_penalty
            score_matrix[i][j] = max(match, delete, insert)
            if score_matrix[i][j] == match:
                traceback[i][j] = 'D'
            elif score_matrix[i][j] == delete:
                traceback[i][j] = 'U'
            else:
                traceback[i][j] = 'L'

    # Traceback to find the alignment
    aligned_seq1 = ''
    aligned_seq2 = ''
    i, j = len(seq1), len(seq2)
    while i > 0 or j > 0:
        if traceback[i][j] == 'D':
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            i -= 1
            j -= 1
        elif traceback[i][j] == 'U':
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = '-' + aligned_seq2
            i -= 1
        else:
            aligned_seq1 = '-' + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            j -= 1

    return score_matrix, aligned_seq1, aligned_seq2

def print_matrix(matrix):
    df = pd.DataFrame(score_matrix)
    print(df.to_string(index=False, header=False))



def print_alignment(seq1, seq2, aligned_seq1, aligned_seq2):
    print("Sequence 1:", seq1)
    print("Sequence 2:", seq2)
    print("Aligned Sequence 1:", aligned_seq1)
    print("Aligned Sequence 2:", aligned_seq2)

seq2 = "AGGGAAATTAACCCCAA"
seq1 = "ATGCCTTAAGGAATT"

score_matrix, aligned_seq1, aligned_seq2 = global_alignment(seq1, seq2)
print("Score Matrix:")
print_matrix(score_matrix)
print("\nBest Possible Alignment:")
print_alignment(seq1, seq2, aligned_seq1, aligned_seq2)

Score Matrix:
  0  -1  -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -13 -14 -15 -16 -17
 -1   1   0 -1 -2 -3 -4 -5 -6 -7  -8  -9 -10 -11 -12 -13 -14 -15
 -2   0   0 -1 -2 -3 -4 -5 -4 -5  -6  -7  -8  -9 -10 -11 -12 -13
 -3  -1   1  1  0 -1 -2 -3 -4 -5  -6  -7  -8  -9 -10 -11 -12 -13
 -4  -2   0  0  0 -1 -2 -3 -4 -5  -6  -7  -6  -7  -8  -9 -10 -11
 -5  -3  -1 -1 -1 -1 -2 -3 -4 -5  -6  -7  -6  -5  -6  -7  -8  -9
 -6  -4  -2 -2 -2 -2 -2 -3 -2 -3  -4  -5  -6  -6  -6  -7  -8  -9
 -7  -5  -3 -3 -3 -3 -3 -3 -2 -1  -2  -3  -4  -5  -6  -7  -8  -9
 -8  -6  -4 -4 -4 -2 -2 -2 -3 -2   0  -1  -2  -3  -4  -5  -6  -7
 -9  -7  -5 -5 -5 -3 -1 -1 -2 -3  -1   1   0  -1  -2  -3  -4  -5
-10  -8  -6 -4 -4 -4 -2 -2 -2 -3  -2   0   0  -1  -2  -3  -4  -5
-11  -9  -7 -5 -3 -4 -3 -3 -3 -3  -3  -1  -1  -1  -2  -3  -4  -5
-12 -10  -8 -6 -4 -2 -3 -2 -3 -4  -2  -2  -2  -2  -2  -3  -2  -3
-13 -11  -9 -7 -5 -3 -1 -2 -3 -4  -3  -1  -2  -3  -3  -3  -2  -1
-14 -12 -10 -8 -6 -4 -2 -2 -1 -2  -3  -2  -2  -3  -4  -4  -3  -2
-15 -13 -11

# Multiple Sequence Alignment


In [16]:
!pip install biopython      #importing the necessary libraries

In [17]:
import numpy as np
import pandas as pd
from Bio import SeqIO
import random

In [18]:
def initial_population(n_initial_population):                                   # the protein sequences in FASTA format were imported in the code
    length = []                                                                 # SeqTO helps read the FASTA files and we can thus align the protein sequences
    sequences = []
    for record in SeqIO.parse("/content/Proteins.txt", "fasta"):
        length.append(len(record.seq))
        sequences.append(str(record.seq))

    max_len = max(length)
    population = []
    for n in range(n_initial_population):
        alig_seq = sequences.copy()
        for s in range(7):
            need_gap = max_len - len(alig_seq[s])
            random_point = random.sample(range(0, max_len), need_gap)
            for p in random_point:
                if len(alig_seq[s]) < max_len:
                    r = random.randint(1, need_gap)
                    alig_seq[s] = alig_seq[s][:p] + ('-' * r) + alig_seq[s][p:]
                    need_gap = max_len - len(alig_seq[s])
                if need_gap == 0:
                  break
        population.append(alig_seq)
    return population, max_len

In [19]:
def calc_fitness(chromosome):
    data = np.loadtxt("/content/PAM250.txt", skiprows=1, dtype='str')
    data = np.array(data[:-1, 1:-1], dtype=int)
    names = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', 'B',
             'J', 'Z', 'X']
    PAM250 = pd.DataFrame(data, index=names, columns=names)

    pairwise_score = []
    for i in range(0, 6):
        for j in range(i + 1, 7):
            seq1 = chromosome[i]
            seq2 = chromosome[j]

            gaps = 0
            weights = 0
            for A, B in zip(seq1, seq2):
                if A == '-' or B == '-':
                    gaps -= 1
                else:
                    weights += PAM250[A][B]
            pairwise_score.append((weights + gaps))

    score = np.sum(pairwise_score)
    return score

In [20]:
def selection(population, n):
    # Roulette Wheel Selection
    selected = []
    fitness_pop = []
    pop_size = len(population)
    for p in range(pop_size):
        fitness_pop.append(calc_fitness(population[p]))
    sum_fitness = sum(fitness_pop)
    normal_fitness = [f / sum_fitness for f in fitness_pop]

    cumulative_fitness = []
    start = 0
    for norm in normal_fitness:
        start += norm
        cumulative_fitness.append(start)
    # print('Cumulativ:',cumulative_fitness)
    for c in range(n):
        r = random.uniform(0, 1)
        individual_number = 0
        for score in cumulative_fitness:
            if r <= score:
                selected.append(population[individual_number])
                break
            individual_number += 1

    return np.array(selected)


In [21]:
def parent_selection(selected):
    return selected[np.random.randint(0, len(selected) - 1, 2)]

In [22]:
def cross_over(parents, length):
    # One Point Crossover
    r = random.randint(2, length - 2)  # --> get random point
    # Set rate 0.8
    if random.random() <= 0.8:
        offspring = {0: [], 1: []}

        # Create First Offspring
        for s1 in range(7):
            ch1_part1 = parents[0][s1][:r]
            ch1_count_without_gap_part1 = 0
            for AA in ch1_part1:
                if AA != '-':
                    ch1_count_without_gap_part1 += 1

            ch1_parent2_AA = str()
            for AA in parents[1][s1]:
                if AA != '-':
                    ch1_parent2_AA += AA
            ch1_parent2_AA = ch1_parent2_AA[ch1_count_without_gap_part1:]
            ch1_count_without_gap_part2 = len(ch1_parent2_AA)

            ch1_count_all_part2 = 0
            reverse_part2 = str()
            for AA in parents[1][s1][::-1]:
                reverse_part2 += AA
                if AA != '-':
                    ch1_count_all_part2 += 1
                if ch1_count_all_part2 == ch1_count_without_gap_part2:
                    break
            ch1_part2 = reverse_part2[::-1]
            remain_len = length - len(ch1_part1 + ch1_part2)
            if remain_len > 0:
                ch1_part2 = ('-' * remain_len) + ch1_part2
            elif remain_len < 0:
                extra = -(remain_len)
                ch1_part2 = ch1_part2.replace('-', '', extra)
            offspring[0].append(ch1_part1 + ch1_part2)

        # Create Second Offspring
        for s2 in range(7):
            ch2_part2 = parents[0][s2][r:]
            ch2_count_without_gap_part2 = 0
            for AA in ch2_part2:
                if AA != '-':
                    ch2_count_without_gap_part2 += 1
            ch2_parent2_AA = str()
            num_AA = 0
            for AA in parents[1][s2]:
                if AA != '-':
                    ch2_parent2_AA += AA
                    num_AA += 1
            need_AA_part1 = num_AA - ch2_count_without_gap_part2
            pr2 = ch2_parent2_AA[:need_AA_part1]
            ch2_count_without_gap_part1 = len(pr2)

            ch2_part1 = str()
            count_p1 = 0
            for AA in parents[1][s2]:
                ch2_part1 += AA
                if AA != '-':
                    count_p1 += 1
                if count_p1 == ch2_count_without_gap_part1:
                    break
            need_len = length - len(ch2_part1 + ch2_part2)
            if need_len > 0:
                ch2_part1 = ch2_part1 + ('-' * need_len)
            elif need_len < 0:
                extra = -(need_len)
                ch2_part1 = ch2_part1.replace('-', '', extra)

            offspring[1].append(ch2_part1 + ch2_part2)
        # Return Best (fitness) Child
        child1_fit = calc_fitness(offspring[0])
        child2_fit = calc_fitness(offspring[1])
        if child1_fit > child2_fit:
            return offspring[0]
        else:
            return offspring[1]

    else:
        # Return a random Parent as Child
        rand_parent = random.randint(0, 1)
        return parents[rand_parent]

In [23]:
def mutation(offspring):
    # Swap mutation
    if random.random() <= 0.2:
        mutate_offspring = []
        for s in range(7):
            child = list(offspring[s])
            for i in range(len(child) - 1):
                AA_i = child[i]
                AA_i_1 = child[i + 1]
                if AA_i != '-' and AA_i_1 == '-':
                    child[i + 1] = AA_i
                    child[i] = AA_i_1
                    break
            mutate_offspring.append(''.join(child))
        return mutate_offspring
    else:
        return offspring

In [24]:
def update_population(population, n):
    population = np.array(population)
    fitness = np.zeros(len(population), dtype=np.int32)
    for idx, ch in enumerate(population):
        fitness[idx] = calc_fitness(ch)
    argsort_fitness = np.argsort(fitness)[-n:]
    return population[argsort_fitness].tolist()

In [25]:
if __name__ == '__main__':
    n_population = 50
    population, length = initial_population(n_population)
    iteration = 10
    n_selection = 40
    print('Iteration:')
    for i in range(iteration):
        print('\t', i)
        selected = selection(population, n_selection)
        childs = []
        for _ in range(0, len(selected), 2):
            parents = parent_selection(selected)
            offspring = cross_over(parents, length)
            mutate_offspring = mutation(offspring)

            childs += [mutate_offspring]
        population += childs
        population = update_population(population, n_population)

    fitness_pop = []
    for chrom in population:
        fitness_pop.append(calc_fitness(chrom))

    best_match_idx = np.where(fitness_pop == np.max(fitness_pop))[0][0]

    print('\n Best multiple alignment:')
    # final = []
    for line in population[best_match_idx]:
        print(line)

    print('Best fitness:', fitness_pop[best_match_idx])

Iteration:
	 0
	 1
	 2
	 3
	 4
	 5
	 6
	 7
	 8
	 9

 Best multiple alignment:
MNSLVSWQLLLFLCATHFGEPLEKVASVGNSRPTGQQ-L-ESLGLLAPGEQSLPCTERKPAATARLSR--------RGTSLSPPPESSGSP-QQPGLSAPHSRQIPAPQGAVLVQREKDLPNYNWNSFGLRFGKREAAPGNHGRSAGRG
MISMASWQLLLLLCVATYGEPLAKVKPGSTGQQSGPQELVNAWEKESRYA-E----------------SKPGSAGLRARRSSPCPPVEGPAGRQRPLCASRSRLIPAPRGA---VLVQREKDLSTYNW---NSFGLRYGRRQAARAARG
MMLLTVILMLSVARVHTNPSGHFQYYLEDETPEETSLRVLRGTDTRPTDGSPPSKLSALFSMGAGPQ-K----NTWWWSPESPYTKRRQNVA----------------------YYNLNSFGLRYGKREQDMLTRLKQKSPVK------
MSSLCLFLFLLGIHLGRSDHTAKNTDELY-SQVPGKSQWLGS------------------LLCPEKVPTTRRAEQMPVLSLLCRRKKSLSTGHPWSTDSLLPSRSISAPEGEF-LVQREKDLSTYNWNSFGLRYGKRGSGSENSKTKVW
MRSSVYWQLLLLLSVSPFGETSDKFAPVENPGRTGTSSLSQEASSSDFSALGQPGRLLAHLIPWEGRPQCLEKPEQTGQTQRLAMLCPSDEASDPLWPGLCPTRSRLITAPQGALLVEREKDMSTYNWNSFGLRYGKRQTNRGKARVPA
MNTRALIL-FMSAMVSQSTAMRAILTDMDTPEPMPDPKPRFLSMERRQFEEPSASDDASLCFFIQEKD---------ETSQISCKHRLARSKFNYNPFGL--------------RFGKRNEATTSDSDRLKHKHLLPMMLYLRKQLETS
MLLLLLLTLVISQHAVGGTMFR